In [5]:
import numpy as np
from collections import defaultdict

In [6]:
def convert_to_links_dict(edge_list):
    links = defaultdict(list)
    for src, dest in edge_list:
        links[src].append(dest)
        links[dest]
    return links

In [7]:
def pagerank(links, alpha=0.85, max_iter=100):
    n = len(links)
    S = np.zeros((n, n))

    for i, src in enumerate(links):
        if len(links[src]) == 0:
            S[i] = np.ones(n) / n
        else:
            for dest in links[src]:
                j = list(links.keys()).index(dest)
                S[i, j] = 1 / len(links[src])
    
    pr = np.random.dirichlet(np.ones(n), 1).T
    G = alpha * S.T + (1 - alpha) / n

    for _ in range(max_iter):
        pr = G @ pr
    
    pagerank_scores = {node:pr[i, 0] for i, node in enumerate(links)}
    return sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)

In [8]:
def hits(links, max_iter=100):
    n = len(links)
    S = np.zeros((n, n))

    for i, src in enumerate(links):
        for dest in links[src]:
            j = list(links.keys()).index(dest)
            S[i, j] = 1

    hubs = np.ones((n, 1)) / n
    auth = np.ones((n, 1)) / n

    for _ in range(max_iter):
        last_hubs = hubs
        last_auth = auth

        hubs = S @ last_auth
        auth = S.T @ last_hubs

    hubs /= np.sum(hubs)
    auth /= np.sum(auth)

    hubs_scores = {node:hubs[i, 0] for i, node in enumerate(links)}
    auth_scores = {node:auth[i, 0] for i, node in enumerate(links)}
    return sorted(hubs_scores.items(), key=lambda x: x[1], reverse=True), sorted(auth_scores.items(), key=lambda x: x[1], reverse=True)

In [9]:
edge_list = [('A', 'B'), ('B', 'C'), ('C', 'A'), ('A', 'D')]

# Convert the edge list to a links dictionary
# links = {
#     'A': ['B', 'D'],
#     'B': ['C'],
#     'C': ['A'],
#     'D': []
# }

links = convert_to_links_dict(edge_list)

pr_scores = pagerank(links)
hub_scores, auth_scores = hits(links)

for node, rank in pr_scores:
    print(f"Node {node} has PageRank: {rank}")
print()
for node, hub in hub_scores:
    print(f"Node {node} has HITS hub: {hub}")
print()
for node, authority in auth_scores:
    print(f"Node {node} has HITS authority: {authority}")

Node A has PageRank: 0.3078534031413612
Node C has PageRank: 0.2646222887060583
Node B has PageRank: 0.21376215407629018
Node D has PageRank: 0.21376215407629018

Node A has HITS hub: 0.9999999999999982
Node B has HITS hub: 8.881784197001237e-16
Node C has HITS hub: 8.881784197001237e-16
Node D has HITS hub: 0.0

Node B has HITS authority: 0.49999999999999956
Node D has HITS authority: 0.49999999999999956
Node A has HITS authority: 4.440892098500622e-16
Node C has HITS authority: 4.440892098500622e-16
